# Improvement Carvana_local_learning_prod_ready

## Подключение библиотек и загрузка датасета

In [ ]:
# Улучшения: 
# Убрал .cuda() из класса модели и добавил в даталоадеры перенос на gpu
# Добавил трэйсинг моделии и ее сохранение, загрузку модели
# Заменил лишний ресайз масок на передачу оригинолов масок
# Сделал предсказания с разным размером батча
# Нейтрализовал проявления хардкода

In [ ]:
# Написать комментарии и пояснения
# Реализовать аугментацию
# Разобраться как лучше интерполировать или ресайзить и как лучше интерполировать
# Сделать submission при обучении на всем трерировочном датасете
# Нужна ли стандартизация вместо нормализации
# Попробовать softdice loss + bce (как в dlcource.ai)

In [ ]:
!pip install segmentation_models_pytorch

In [1]:
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import numpy as np
import glob
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
import time
from torch.autograd import Variable

In [ ]:
# Выполнять, если датасет не загружен
!pip install -q kaggle
!mkdir ~/.kaggle
!cp ~/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c carvana-image-masking-challenge
!unzip ~/carvana-image-masking-challenge.zip ~/carvana_dataset/

!unzip ~/carvana_dataset/train.zip -d ~/carvana_dataset/train
!unzip ~/carvana_dataset/test.zip -d ~/carvana_dataset/test
!unzip ~/carvana_dataset/train_masks.zip -d ~/carvana_dataset/train_masks

!unzip ~/carvana_dataset/train_hq.zip -d ~/carvana_dataset/train_hq
!unzip ~/carvana_dataset/test_hq.zip -d ~/carvana_dataset/test_hq

!unzip ~/carvana_dataset/train_masks.csv.zip  ~/carvana_dataset/
!unzip ~/carvana_dataset/sample_submission.csv.zip  ~/carvana_dataset/
!unzip ~/carvana_dataset/metadata.csv.zip  ~/carvana_dataset/

!rm ~/carvana-image-masking-challenge.zip
!rm ~/carvana_dataset/test.zip
!rm ~/carvana_dataset/train_masks.zip
!rm ~/carvana_dataset/train.zip
!rm ~/carvana_dataset/test_hq.zip
!rm ~/carvana_dataset/train_hq.zip
!rm ~/carvana_dataset/train_masks.csv.zip
!rm ~/carvana_dataset/sample_submission.csv.zip
!rm ~/carvana_dataset/metadata.csv.zip

In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

## Используемые функции

In [2]:
def get_data_csv(imgs_path: str = None, masks_path: str = None) -> pd.DataFrame:
    '''Функция получает на вход пути к директориям с изображениями и масками
    и генерирует датафрейм, содержащий имя изображений, их адреса и адреса
    соответствующих им масок
  
    Входные параметры:
    imgs_path: str - путь к директории с изображениями,
    masks_path: str - путь к директории с масками
    Выходные параметры:
    pd.DataFrame: data - dataframe, содержащий адреса изображений и соответствующих им масок'''

    assert (imgs_path != None) & (masks_path != None)
    # imgs_path or masks_path is equal None

    data_img = {}
    data_mask = {}
    data_img['imgs_path'] = []
    data_mask['masks_path'] = []
    data_img['imgs_path'] = list(glob.glob(imgs_path + "/*"))
    data_mask['masks_path'] = list(glob.glob(masks_path + "/*"))

    data_img = pd.DataFrame(data_img)
    data_mask = pd.DataFrame(data_mask)

    def file_name(x):
        return x.split("/")[-1].split(".")[0]

    data_img["file_name"] = data_img["imgs_path"].apply(lambda x: file_name(x))
    data_mask["file_name"] = data_mask["masks_path"].apply(lambda x: file_name(x)[:-5])

    data = pd.merge(data_img, data_mask, on = "file_name", how = "inner")

    return data

In [3]:
def get_train_test(source_df: pd.DataFrame, separate_feature: str = None, test_size: float = 0.25) -> pd.DataFrame:
    '''Функция разделяет source_df на две части с коэффициентом test_size
    по уникальным значениям separate_feature так, чтобы в новых датафреймах
    не было строк с одинаковыми значенияти из separate_feature

    Входные параметры:
    source_df: pd.DataFrame - датафрейм для разделения на train и test
    separate_feature: str - поле, по которому датафрейм будет разделен
    test_size: float - коэффициент разделения дтафрейма
    Выходные параметры:
    pd.DataFrame: data_train - датафрейм для тренировки
    pd.DataFrame: data_valid - датафрейм для валидации'''
  
    if (separate_feature != None) & (separate_feature in source_df.columns):
        train_cars, valid_cars = train_test_split(source_df[separate_feature].unique(), test_size=test_size, random_state=42)
        data_valid = source_df[np.isin(source_df[separate_feature].values, valid_cars)]
        data_train = source_df[np.isin(source_df[separate_feature].values, train_cars)]
        assert data.shape[0] == (data_valid.shape[0] + data_train.shape[0])
        assert np.isin(data_train[separate_feature].values, data_valid[separate_feature].values).sum() == 0
    else:
        data_train, data_valid = train_test_split(source_df, test_size=test_size)

    return data_train, data_valid


In [4]:
def DICE(logits: torch.Tensor, targets: torch.Tensor, treashold: float) -> float:
    '''Функция для вычисления DICE коэффициента для набора изображенй в формате torch.Tensor
    Входные параметры:
    logits: torch.Tensor - тензор из предсказанных масок в logit масштабе
    targets: torch.Tensor - тензор из целевых целевых значений масок
    treashold: float - порог для определения класса точки в предсказанной точке
    Выходные параметры:
    score: float - значение DICE коэффициента для набора предсказанных масок'''
    
    smooth = 1
    num = targets.size(0)
    probs = torch.sigmoid(logits)
    outputs = torch.where(probs > treashold, 1, 0)
    m1 = outputs.view(num, -1)
    m2 = targets.view(num, -1)
    intersection = (m1 * m2)

    score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
    score = score.sum() / num
    return score

In [5]:
def tensor_to_rle(tensor: torch.Tensor) -> str:
    '''Функция принимает одну маску в тензорном формате, элементы которой
    имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
    Входные параметры:
    tensor: torch.Tensor - маска в тензорном формате
    Выходные параметры:
    rle_str: str - rle представление маски в строком виде'''
    
    # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
    # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
    # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
    # не сильно упадет)
    tensor = tensor.view(1, -1)
    tensor = tensor.squeeze(0)
    tensor[0] = 0
    tensor[-1] = 0
    rle = torch.where(tensor[1:] != tensor[:-1])[0] + 2
    rle[1::2] = rle[1::2] - rle[:-1:2]
    rle = rle.cpu().detach().numpy()
    rle_str = rle_to_string(rle)
    return rle_str

In [6]:
def numpy_to_rle(mask_image: np.ndarray) -> str:
    '''Функция принимает одну маску в формате массива numpy, элементы которой
    имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
    Входные параметры:
    mask_image: numpy.ndarray - маска в тензорном формате
    Выходные параметры:
    rle_str: str - rle представление маски в строком виде'''
    
    # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
    # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
    # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
    # не сильно упадет)
    pixels = mask_image.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle_str = rle_to_string(runs)
    return rle_str

In [7]:
def rle_to_string(runs: torch.Tensor) -> str:
    '''Функция преобразует последовательноть чисел в тензоре runs
    в строковое представление этой последовательности
    Входные параметры:
    runs: torch.Tensor - последовательность чисел в тензорном формате
    Выходные параметры:
    rle_str: str - строковое представление последовательности чисел'''
    
    return ' '.join(str(x) for x in runs)

In [8]:
def mask_to_rle(mask_addr: str) -> str:
    '''Функция преобразует маску, имеющую адрес mask_addr и сохраненную в
    формате .gif, элементы которой имеют значения 0 и 1 в rle представление
    в строковом виде'''
    
    mask = Image.open(mask_addr).convert('LA') # преобразование в серый
    mask = np.asarray(mask).astype('float')[:,:,0]
    mask = mask/255.0
    mask_rle = numpy_to_rle(mask)
    return mask_rle

## Используемые классы

In [9]:
class DiceMetric(nn.Module):
    def __init__(self, treashold=0.5):
        super(DiceMetric, self).__init__()
        self.treashold = treashold

    def forward(self, logits, targets):
        with torch.no_grad():
            smooth = 1
            num = targets.size(0)
            probs = torch.sigmoid(logits)
            outputs = torch.where(probs > self.treashold, 1, 0)
            m1 = outputs.view(num, -1)
            m2 = targets.view(num, -1)
            intersection = (m1 * m2)

            score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
            score = score.sum() / num
            return score

In [10]:
class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits, targets):
        smooth = 1
        num = targets.size(0)
        probs = torch.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score

In [11]:
class CustomDatasetForTrain(Dataset):
    def __init__(self, data_info, device, out_shape=(512, 512), skip_mask=False):
        # Подаем наш подготовленный датафрейм
        self.data_info = data_info
        # Разделяем датафрейм на rgb картинки 
        self.image_arr = self.data_info.iloc[:,0]
        # и на сегментированные картинки
        self.mask_arr = self.data_info.iloc[:,2]
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
        # Устройство, на котором будут находиться выходные тензоры
        self.device = device
        # Пространственные размеры тензоров на выходе объекта даталоадера
        self.out_shape = out_shape
        # Нужно ли пробрасывать маску изображения на выход без изменений
        self.skip_mask = skip_mask

    def __getitem__(self, index):
        img = np.asarray(Image.open(self.image_arr[index])).astype('float')
        img = (torch.as_tensor(img)/255.0).to(self.device)
        # unsqueeze - чтобы interpolate работало
        # permute - переставляем измерение каналов на 2-е место
        img = img.unsqueeze(0).permute(0,3,1,2)
        # clamp не позволяет выйти за границы значений
        img = F.interpolate(input=img, size=self.out_shape, align_corners=False, mode='bicubic').clamp(min=0, max=1)
        img = img.squeeze(0)
        # Маски - одноканальные изображения со значениями 0 и 1
        mask = Image.open(self.mask_arr[index])
        mask = np.asarray(mask).astype('float')
        # unsqueeze - добавляем измерение каналов
        mask = (torch.as_tensor(mask)).to(self.device).unsqueeze(0) 
        # unsqueeze - чтобы interpolate работало
        mask_small = mask.unsqueeze(0)
        mask_small = F.interpolate(input=mask_small, size=self.out_shape, mode='nearest')
        mask_small = mask_small.squeeze(0)
        
        # Если необходима исходная маска, то дополнительно возвращаем ее
        if self.skip_mask == True:
            return (img.float(), mask_small.float(), mask.float())
        else:
            return (img.float(), mask_small.float())

    def __len__(self):
        return self.data_len

In [12]:
class CustomDatasetForTest(Dataset):
    def __init__(self, data_info, out_shape=(512, 512)):
        # Подаем наш подготовленный датафрейм
        self.data_info = data_info
        
        # Получаем адреса RGB изображений 
        self.image_addresses = self.data_info.iloc[:,0]
        
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
        
        self.out_shape = out_shape

    def __getitem__(self, index):
        img = np.asarray(Image.open(self.image_addresses[index])).astype('float')
        img = torch.as_tensor(img)/255    
        # unsqueeze - чтобы interpolate работало
        # permute - переставляем измерение каналов на 2-е место
        img = img.unsqueeze(0).permute(0,3,1,2)
        # clamp не позволяет выйти за границы значений
        img = F.interpolate(input=img, size=self.out_shape, align_corners=False, mode='bicubic').clamp(min=0, max=1)
        img = img.squeeze(0)
        
        image_address = self.image_addresses[index]
        image_name = image_address.split('/')[-1]
    
        return (index, img.float(), image_name)

    def __len__(self):
        return self.data_len

In [36]:
# Попробовать разные типы интерполяции

class NeuralNetwork(nn.Module):
    def __init__(self, model):
        super(NeuralNetwork, self).__init__()
        self.model = model

    def forward(self, x):
        x = self.model(x)
        return x
    
    @staticmethod
    def tensor_to_rle(tensor):
        # We avoid issues with '1' at the start or end (at the corners of 
        # the original image) by setting those pixels to '0' explicitly.
        # We do not expect these to be non-zero for an accurate mask, 
        # so this should not harm the score.
        with torch.no_grad():
            tensor = tensor.view(1, -1)
            tensor = tensor.squeeze(0)
            tensor[0] = 0
            tensor[-1] = 0
            rle = torch.where(tensor[1:] != tensor[:-1])[0] + 2
            rle[1::2] = rle[1::2] - rle[:-1:2]
            rle = rle.cpu().detach().numpy()
            rle_str = rle_to_string(rle)
            return rle_str
    
    @staticmethod
    def rle_to_string(runs):
        return ' '.join(str(x) for x in runs)
    
    
    
    def fit(self, criterion, metric, optimizer, train_data_loader, valid_data_loader=None, epochs=1):
        
        self.optimizer = optimizer
        # запускаем главный тренировочный цикл
        epoch_train_losses = []
        epoch_valid_losses = []
        epoch_valid_metrics = []
        result = {}
        
        for epoch in range(epochs):
            self.model.train()
            time1 = time.time()
            running_loss =0.0
            train_losses = []
            for batch_idx, (data, labels) in enumerate(train_data_loader):
                data, labels = Variable(data), Variable(labels)        

                optimizer.zero_grad()
                outputs = self.model(data)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                train_losses.append(loss.item())
                if (batch_idx+1) % 300 == 299:
                    print(f'Train Epoch: {epoch+1}, Loss: {running_loss/300}')
                    time2 = time.time()
                    print(f'Spend time for 300 batches: {time2-time1} sec')
                    time1 = time.time()
                    running_loss = 0.0

            train_loss = np.mean(train_losses)        
            
            
            if valid_data_loader != None:
                self.model.eval()
                valid_metrics = []
                valid_losses = []
                for batch_idx, (data, labels_small, labels) in enumerate(valid_data_loader):
                    data, labels, labels_small = Variable(data), Variable(labels), Variable(labels_small)
                    outputs = self.model(data)
                    # loss вычисляется для сжатых масок для правильной валидации (обучались на сжатых)
                    # чтобы вовремя определить переобучение
                    loss = criterion(outputs, labels_small)
                    valid_losses.append(loss.item())
                    #Преобразуем выход модели к размеру соответствующей маски
                    outputs = F.interpolate(input=outputs, size=(labels.shape[2], labels.shape[3]), mode='nearest')

                    # метрика считается для исходных размеров потому что именно так итоговое качество
                    # определяется алгоритмом kaggle 
                    metric_value = metric(outputs, labels)
                    valid_metrics.append(metric_value.item())
                    
                valid_loss    = np.mean(valid_losses)
                valid_metric  = np.mean(valid_metrics)
                print(f'Epoch {epoch+1}, train loss: {train_loss}, valid_loss: {valid_loss}, valid_metric: {valid_metric}')
            else:
                print(f'Epoch {epoch+1}, train loss: {train_loss}')
                valid_loss = None
                valid_metric = None
            
            epoch_train_losses.append(train_loss)
            epoch_valid_losses.append(valid_loss)
            epoch_valid_metrics.append(valid_metric)
        
        result['epoch_train_losses'] = epoch_train_losses
        result['epoch_valid_losses'] = epoch_valid_losses
        result['epoch_valid_metrics'] = epoch_valid_metrics
        
        return result
    
    
    
    def predict(self, test_data_loader, predict_directory, output_size=(1280, 1918), 
                                                           mask_treashold=0.5, generate_rle_dataframe=True):
        self.model.eval()
        img_names = []
        img_rles = []
        
        for batch_idx, (index, img, img_name)  in enumerate(test_data_loader):

            img = Variable(img)        
            img = img.cuda()
            pred_mask_logit = self.model(img)
            pred_mask_logit = F.interpolate(input=pred_mask_logit, size=output_size, mode='nearest')
            pred_mask_logit_prob = torch.sigmoid(pred_mask_logit)
            pred_mask = torch.where(pred_mask_logit_prob > mask_treashold, 1, 0)
            #pred_mask = pred_mask.squeeze(0)
            
            # Каждое изображение в тензоре преобразуем в картинку и сохраняем
            for i in range(pred_mask.shape[0]):
                mask = (pred_mask[i].cpu().numpy() * 255.0)[0] # [0] - избавляемся от батч размерности
                PIL_image = Image.fromarray(mask.astype('uint8'), 'L')
                PIL_image.save((predict_directory+img_name[i]).split('.')[0]+'.gif')
                
                # Если требуется, получаем значения rle для каждой картинки
                if generate_rle_dataframe == True:
                    img_names.append(img_name[i])
                    img_rles.append(tensor_to_rle(pred_mask[i]))
                
        if generate_rle_dataframe == True:
            rle_dataframe = pd.DataFrame(list(zip(img_names, img_rles)), columns =['img_name', 'img_rle'])
            return rle_dataframe
    
    def save(self, path_to_save='./model.pth'):
        torch.save(self.model.state_dict(), path_to_save)
    
    def trace_save(self, path_to_save='./model.pth'):
        example_forward_input = torch.rand(1, 3, 512, 512).to('cpu')
        if next(self.model.parameters()).is_cuda:
            example_forward_input= example_forward_input.to('cuda:0')
            
        traced_model = torch.jit.trace(self.model, example_forward_input)
        torch.jit.save(traced_model, 'model.pt')
    
    def load(self, path_to_model='./model.pth'):
        self.model.load_state_dict(torch.load(path_to_model))

## Обучение модели

In [37]:
dataset_path = '/home/dima/carvana_dataset'
imgs_path  = dataset_path + '/train/train'
masks_path = dataset_path + '/train_masks/train_masks'

nn_image_shape = (512, 512)
learning_rate = 0.001
num_epochs = 5
mask_treashold = 0.5

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [38]:
data = get_data_csv(imgs_path=imgs_path, masks_path=masks_path)
    
# Добавляем признак, по которому будем разбивать датасет на train и test,
# чтобы не было разных фотографий одной и той же машины в двух датасетах
data["car"] = data["file_name"].apply(lambda x: x.split('_')[0])

In [25]:
# Обучение с валидацией
train_df, valid_df = get_train_test(data, separate_feature='car', test_size=0.25)
train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)

train_data = CustomDatasetForTrain(train_df, device, out_shape=nn_image_shape)
valid_data = CustomDatasetForTrain(valid_df, device, out_shape=nn_image_shape, skip_mask=True)

train_data_loader = DataLoader(train_data,batch_size=4,shuffle=True)
valid_data_loader = DataLoader(valid_data,batch_size=4,shuffle=False)

In [39]:
# Обучение без валидации
train_data = CustomDatasetForTrain(data, device, out_shape=nn_image_shape)
train_data_loader = DataLoader(train_data,batch_size=4, shuffle=True)

In [40]:
# Создаем модель на основе предложенной архитектуры
model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
                 encoder_weights='imagenet', decoder_channels = [256, 128, 64, 32, 16]).to(device)

#model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
#                 encoder_weights='imagenet').to(device)

my_model = NeuralNetwork(model=model)

In [41]:
criterion = SoftDiceLoss()
optimizer = torch.optim.Adam(my_model.parameters(), lr=learning_rate)
metric = DiceMetric(treashold=mask_treashold)

In [42]:
my_model.fit(criterion,
             metric,
             optimizer,
             train_data_loader,
             #valid_data_loader,
             epochs=num_epochs)

Train Epoch: 1, Loss: 0.05624229907989502
Spend time for 300 batches: 102.3834879398346 sec
Train Epoch: 1, Loss: 0.009145987232526144
Spend time for 300 batches: 103.09858179092407 sec
Train Epoch: 1, Loss: 0.011683658560117086
Spend time for 300 batches: 103.74246835708618 sec
Train Epoch: 1, Loss: 0.006861371596654256
Spend time for 300 batches: 103.24788451194763 sec
Epoch 1, train loss: 0.02016467460483875
Train Epoch: 2, Loss: 0.0072389378150304155
Spend time for 300 batches: 102.5597140789032 sec
Train Epoch: 2, Loss: 0.006068665981292725
Spend time for 300 batches: 102.10100078582764 sec
Train Epoch: 2, Loss: 0.006499105493227641
Spend time for 300 batches: 101.81708836555481 sec
Train Epoch: 2, Loss: 0.005942440629005432
Spend time for 300 batches: 102.50844478607178 sec
Epoch 2, train loss: 0.006390716435399445
Train Epoch: 3, Loss: 0.005424115657806396
Spend time for 300 batches: 102.09443306922913 sec
Train Epoch: 3, Loss: 0.005184808969497681
Spend time for 300 batches: 10

{'epoch_train_losses': [0.02016467460483875,
  0.006390716435399445,
  0.0061747680107752485,
  0.005132807406989284,
  0.005158050208346649],
 'epoch_valid_losses': [None, None, None, None, None],
 'epoch_valid_metrics': [None, None, None, None, None]}

In [395]:
# Сохраняем веса обученной модели
my_model.save(path_to_save = './model.pth')

In [396]:
# Сохраняем оттрассированную модель
my_model.trace_save(path_to_save = './model.pt')

## Загрузка сохраненной модели

In [16]:
# Воспроизводим модель по известной архитектуре и сохраненным весам
model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
                 encoder_weights='imagenet', decoder_channels = [256, 128, 64, 32, 16]).to(device)

my_model = NeuralNetwork(model=model)
my_model.load(path_to_model = './model.pth')

In [448]:
# Загружаем оттрассированную модель
my_model = torch.jit.load('./model.pt')
my_model = NeuralNetwork(model=my_model)
my_model = my_model.to(device)

## Предсказание модели

In [46]:
predict_directory = '/home/dima/carvana_dataset/test/predict_small/'
test_dataset = '/home/dima/carvana_dataset/test/test/'

In [47]:
test_dataframe = {}
test_dataframe['img_addr'] = list(glob.glob(test_dataset + "/*"))
test_dataframe = pd.DataFrame(test_dataframe)

In [48]:
test_data = CustomDatasetForTest(test_dataframe)
test_data_loader = DataLoader(test_data, batch_size=2, shuffle=False)
#loader = iter(test_data_loader)
#index, img, img_name = loader.next()

In [49]:
rle_dataframe = my_model.predict(test_data_loader, predict_directory, 
                                 mask_treashold=mask_treashold, generate_rle_dataframe=True)

In [50]:
# Получаем датафрейм с результатом для заливки на kaggle
rle_dataframe.to_csv('rle_dataframe.csv', index=True)
sample_submission = pd.read_csv('/home/dima/carvana_dataset/sample_submission.csv')
sample_submission = sample_submission.merge(rle_dataframe, how='left', left_on='img', right_on='img_name')
sample_submission.drop(columns=['rle_mask', 'img_name'], inplace=True)
sample_submission.rename(columns={'img_rle': 'rle_mask'}, inplace=True)
sample_submission.to_csv('submission_04_10.csv', index=False)

In [28]:
torch.cuda.empty_cache()

In [ ]:
# Epoch 5, train loss: 0.006944334619686383, valid_loss: 0.0050198428332805635, valid_metric: 0.9925926834344864